In [148]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

set_label = ['50-25','75-37','100-50','125-67','150-75']
set_A_05 = ['1226_1700', '1226_1713', '1226_1727', '1226_1741', '1226_1756']
set_B_02 = ['0104_2223', '0104_2309', '0104_2328', '0104_2343', '0105_0013']
set_D_06 = ['0104_1949', '0104_2002', '0104_2016', '0104_2032', '0104_2056']

subset = set_B_02
set_name = './imagesets/B_S1_0.08.txt'
# B:8.04% -> 1.1
# D:9.15% -> 0.8

In [149]:
# df = pd.read_csv(s1s2_path)
# X = np.array([df['scan_enp'], df['bev_enp']]).T
# y = np.array(df['s1_gt'])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

def evaluate_regression(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return mse, rmse, mae, r2

In [150]:
idx, scan_entropy_list, bev_entropy_list, pf_scalar_list, pf_vector_list, s1_gt =[], [], [], [], [], []
empty_file = []
sub_cnt = 0
for time_dirc in subset:
    sem_pt_path = '/home/newDisk/tool/carla_dataset_tool/raw_data/record_2024_'+time_dirc+'/vehicle.tesla.model3.master/velodyne_semantic/'
    file_list = os.listdir(sem_pt_path)
    file_list = [f for f in file_list if f.endswith('.txt')]
    file_list.sort()

    for file_name in file_list:
        gt = 0
        with open(sem_pt_path+file_name, 'r') as f:
            lines = f.readlines()
            last_line = lines[-1]
            if len(lines) ==1:
                empty_file.append(str(sub_cnt)+file_name[-9:-4])
            else:
                scores = last_line.split(' ')
                s1_gt.append(scores[0])
                scan_entropy_list.append(np.float64(scores[1]))
                bev_entropy_list.append(np.float64(scores[2]))
                pf_scalar_list.append(np.float64(scores[3]))
                pf_vector_list.append(np.float64(scores[4].replace('\n','')))
                idx.append(str(sub_cnt)+file_name[-9:-4])

    sub_cnt += 1


In [151]:
len(scan_entropy_list)

4658

In [152]:
df = pd.DataFrame({'idx':idx, 'scan_enp':scan_entropy_list, 'bev_enp':bev_entropy_list, 'pf_scalar':pf_scalar_list, 'pf_vector':pf_vector_list, 's1_gt':s1_gt})
X = np.array([df['scan_enp'], df['bev_enp'],df['idx']]).T
y = np.array(df['s1_gt'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

train_idx = X_train[:,-1]
test_idx = X_test[:,-1]
X_train = X_train[:,:-1]
X_test = X_test[:,:-1]

# sns.scatterplot(x=bev_entropy_list, y=s1_gt)

In [153]:
mertic = 1.1
print(len(X_train), len(X_test), mertic)

gbr = GradientBoostingRegressor(loss='huber',learning_rate=0.2,n_estimators=100, min_samples_leaf=10, max_depth=10, max_features=15, random_state=42)
gbr.fit(X_train,y_train)


y_pred_train = gbr.predict(X_train)
gbr_mse, gbr_rmse, gbr_mae, gbr_r2 = evaluate_regression(y_train, y_pred_train)
print('[GBR x and y] test on train dataset')
print('mse: ', gbr_mse)
print('rmse: ', gbr_rmse)
print('mae: ', gbr_mae)
print('r2: ', gbr_r2)

y_pred_test = gbr.predict(X_test)
gbr_mse, gbr_rmse, gbr_mae, gbr_r2 = evaluate_regression(y_test, y_pred_test)
print('[GBR x and y] test on test dataset')
print('mse: ', gbr_mse)
print('rmse: ', gbr_rmse)
print('mae: ', gbr_mae)
print('r2: ', gbr_r2)

# calculate the accuary with metric
y_pred2 = y_pred_test.copy()
y_test2 = y_test.copy()
y_pred2[y_pred2 <= mertic] = 0
y_pred2[y_pred2 > mertic] = 1
y_test2[np.float64(y_test2) <= mertic] = 0
y_test2[np.float64(y_test2) > mertic] = 1
accuracy = np.sum(y_pred2 == y_test2) / len(y_test2)

print(len(y_pred2[y_pred2 == 0]) / len(y_pred2))


print('accuracy: ', accuracy)
        
# get the idx, where y_pred2 == 1 at the same place
idx_test = test_idx[y_pred2 == 1]

print(len(idx_test), len(y_test))


931 3727 1.1


[GBR x and y] test on train dataset
mse:  2.1189078522767715
rmse:  1.4556468844732817
mae:  0.7903050667500368
r2:  0.7627184542263621
[GBR x and y] test on test dataset
mse:  5.312919396041059
rmse:  2.304977092302884
mae:  1.4923082701282764
r2:  0.3897560535700091
0.08049369466058492
accuracy:  0.813254628387443
3427 3727


In [154]:
# frame_id = train_idx + test_idx, write to imageset
train_list = train_idx.tolist() + idx_test.tolist()
train_list.sort()

test_list = []
val_list = []
test_spilt = 12
val_spilt = 15

# idx should not has element in empty_file
for id in empty_file:
    if id in train_list:
        train_list.remove(id)

train_path = './imagesets/B/train.txt'
test_path = './imagesets/B/test.txt'
val_path = './imagesets/B/val.txt'


for i in range(len(train_list)):
    if i % test_spilt == 0:
        test_list.append(train_list[i])
        train_list[i] = 'none'
    if i % val_spilt == 0:
        val_list.append(train_list[i])
        train_list[i] = 'none'

train_list = [x for x in train_list if x != 'none']

with open(train_path, 'w') as f:
    for item in train_list:
        f.write("%s\n" % item)

with open(test_path, 'w') as f:
    for item in test_list:
        f.write("%s\n" % item)

with open(val_path, 'w') as f:
    for item in val_list:
        f.write("%s\n" % item)